In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('train (2).csv')

In [4]:
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
344,345,0,2,"Fox, Mr. Stanley Hubert",male,36.0,0,0,229236,13.0000,NaN,S
653,654,1,3,"O'Leary, Miss. Hanora ""Norah""",female,NaN,0,0,330919,7.8292,NaN,Q
411,412,0,3,"Hart, Mr. Henry",male,NaN,0,0,394140,6.8583,NaN,Q


Let's Plan

In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [6]:
# step 1 -> train/test/split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],
                                                test_size=0.2,
                                                random_state=0)

In [39]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S


In [34]:
# y_train


In [9]:
# imputation transformer
tnf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [35]:
tnf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [10]:
#  one hot encoding
tnf2 = ColumnTransformer([
    ('ohe_sex',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1]),
    ('ohe_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[6])
],remainder='passthrough')

In [36]:
tnf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [1]),
                                ('ohe_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [6])])

In [11]:
# Scaling
tnf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [12]:
#  Feature Selection
tnf4 = SelectKBest(score_func=chi2,k=8)

In [13]:
#  train the model
tnf5 = DecisionTreeClassifier()

CREATE PIPELINE

In [14]:
pipe= Pipeline([
    ('tnf1',tnf1),
    ('tnf2',tnf2),
    ('tnf3',tnf3),
    ('tnf4',tnf4),
    ('tnf5',tnf5)
])

Pipeline Vs make_pipeline

Pipeline requires naming of steps, make_pipeline does not.
(same applies to ColumnTransformer Vs make_column_transformer)

In [15]:
# Alternate Syntax
pipe = make_pipeline(tnf1,tnf2,tnf3,tnf4,tnf5)

In [16]:
#  train
pipe.fit(x_train,y_train)

C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1]),
                                                 ('ohe_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000017E118FBEC0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

Explore the Pipeline

In [17]:
# code here
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [18]:
#  Display Pipeline

from sklearn import set_config
set_config(display = 'diagram')

PREDICTION TIME

In [19]:
y_pred = pipe.predict(x_test)
y_pred.shape

(179,)

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
accuracy_score(y_test,y_pred)

0.6759776536312849

CROSS VALIDATION USING PIPELINE

In [22]:
# cross validation using croos_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()

C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse

0.6152171771890081

GridSearch using Pipeline

In [23]:
#  gridsearchcv
params = {
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}

In [24]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Atharva\ANACONDAS\envs\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex',
                                                                         OneHotEncoder(handle_unknown='ignor...
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000017E118FBEC0>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [25]:
grid.best_score_

0.6152171771890081

In [26]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 5}

Exporting the Pipeline

In [27]:
# export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))